# Tekstklassifikationsopgave

Som nævnt vil vi fokusere på en simpel tekstklassifikationsopgave baseret på **AG_NEWS**-datasættet, som går ud på at klassificere nyhedsoverskrifter i en af 4 kategorier: Verden, Sport, Erhverv og Videnskab/Teknologi.

## Datasættet

Dette datasæt er indbygget i [`torchtext`](https://github.com/pytorch/text)-modulet, så vi kan nemt få adgang til det.


In [1]:
import torch
import torchtext
import os
import collections
os.makedirs('./data',exist_ok=True)
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
classes = ['World', 'Sports', 'Business', 'Sci/Tech']

Her indeholder `train_dataset` og `test_dataset` samlinger, der returnerer par af label (nummer på klasse) og tekst henholdsvis, for eksempel:


In [2]:
list(train_dataset)[0]

(3,
 "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.")

Så lad os udskrive de første 10 nye overskrifter fra vores datasæt:


In [5]:
for i,x in zip(range(5),train_dataset):
    print(f"**{classes[x[0]]}** -> {x[1]}")


**Sci/Tech** -> Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.
**Sci/Tech** -> Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\which has a reputation for making well-timed and occasionally\controversial plays in the defense industry, has quietly placed\its bets on another part of the market.
**Sci/Tech** -> Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\about the economy and the outlook for earnings are expected to\hang over the stock market next week during the depth of the\summer doldrums.
**Sci/Tech** -> Iraq Halts Oil Exports from Main Southern Pipeline (Reuters) Reuters - Authorities have halted oil export\flows from the main pipeline in southern Iraq after\intelligence showed a rebel militia could strike\infrastructure, an oil official said on Saturday.
**Sci/Tech** -> Oil prices soar to

Fordi datasæt er iteratorer, hvis vi vil bruge dataene flere gange, skal vi konvertere det til en liste:


In [3]:
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
train_dataset = list(train_dataset)
test_dataset = list(test_dataset)

## Tokenisering

Nu skal vi konvertere tekst til **tal**, som kan repræsenteres som tensorer. Hvis vi ønsker repræsentation på ordniveau, skal vi gøre to ting:
* bruge en **tokenizer** til at opdele teksten i **tokens**
* opbygge et **ordforråd** af disse tokens.


In [4]:
tokenizer = torchtext.data.utils.get_tokenizer('basic_english')
tokenizer('He said: hello')

['he', 'said', 'hello']

In [5]:
counter = collections.Counter()
for (label, line) in train_dataset:
    counter.update(tokenizer(line))
vocab = torchtext.vocab.vocab(counter, min_freq=1)

Ved hjælp af ordforråd kan vi nemt kode vores tokeniserede streng til et sæt tal:


In [19]:
vocab_size = len(vocab)
print(f"Vocab size if {vocab_size}")

stoi = vocab.get_stoi() # dict to convert tokens to indices

def encode(x):
    return [stoi[s] for s in tokenizer(x)]

encode('I love to play with my words')

Vocab size if 95810


[599, 3279, 97, 1220, 329, 225, 7368]

## Bag of Words tekstrepræsentation

Fordi ord repræsenterer betydning, kan vi nogle gange forstå meningen med en tekst blot ved at se på de enkelte ord, uanset deres rækkefølge i sætningen. For eksempel, når vi klassificerer nyheder, er ord som *vejr*, *sne* sandsynligvis indikatorer for *vejrudsigter*, mens ord som *aktier*, *dollar* ville pege på *finansielle nyheder*.

**Bag of Words** (BoW) vektorrepræsentation er den mest almindeligt anvendte traditionelle vektorrepræsentation. Hvert ord er knyttet til en vektorindeks, og vektorelementet indeholder antallet af forekomster af et ord i et givet dokument.

![Billede, der viser, hvordan en bag of words-vektorrepræsentation er repræsenteret i hukommelsen.](../../../../../translated_images/da/bag-of-words-example.606fc1738f1d7ba9.webp) 

> **Note**: Du kan også tænke på BoW som en sum af alle one-hot-kodede vektorer for de enkelte ord i teksten.

Nedenfor er et eksempel på, hvordan man genererer en bag of words-repræsentation ved hjælp af Scikit Learn python-biblioteket:


In [7]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[1, 1, 0, 2, 0, 0, 0, 0, 0]], dtype=int64)

For at beregne bag-of-words-vektoren fra vektorrepræsentationen af vores AG_NEWS-datasæt, kan vi bruge følgende funktion:


In [20]:
vocab_size = len(vocab)

def to_bow(text,bow_vocab_size=vocab_size):
    res = torch.zeros(bow_vocab_size,dtype=torch.float32)
    for i in encode(text):
        if i<bow_vocab_size:
            res[i] += 1
    return res

print(to_bow(train_dataset[0][1]))

tensor([2., 1., 2.,  ..., 0., 0., 0.])


> **Bemærk:** Her bruger vi den globale `vocab_size`-variabel til at angive standardstørrelsen på ordforrådet. Da ordforrådsstørrelsen ofte er ret stor, kan vi begrænse størrelsen på ordforrådet til de mest hyppige ord. Prøv at sænke værdien af `vocab_size` og køre koden nedenfor, og se hvordan det påvirker nøjagtigheden. Du bør forvente et vist fald i nøjagtighed, men ikke dramatisk, til fordel for højere ydeevne.


## Træning af BoW-klassifikator

Nu hvor vi har lært, hvordan man opbygger en Bag-of-Words-repræsentation af vores tekst, lad os træne en klassifikator ovenpå den. Først skal vi konvertere vores datasæt til træning på en måde, så alle positionelle vektorrepræsentationer bliver konverteret til Bag-of-Words-repræsentation. Dette kan opnås ved at bruge `bowify`-funktionen som `collate_fn`-parameter til standard torch `DataLoader`:


In [21]:
from torch.utils.data import DataLoader
import numpy as np 

# this collate function gets list of batch_size tuples, and needs to 
# return a pair of label-feature tensors for the whole minibatch
def bowify(b):
    return (
            torch.LongTensor([t[0]-1 for t in b]),
            torch.stack([to_bow(t[1]) for t in b])
    )

train_loader = DataLoader(train_dataset, batch_size=16, collate_fn=bowify, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, collate_fn=bowify, shuffle=True)

Lad os nu definere et simpelt klassifikations-neuralt netværk, der indeholder et lineært lag. Størrelsen på inputvektoren er lig med `vocab_size`, og outputstørrelsen svarer til antallet af klasser (4). Da vi løser en klassifikationsopgave, er den endelige aktiveringsfunktion `LogSoftmax()`.


In [22]:
net = torch.nn.Sequential(torch.nn.Linear(vocab_size,4),torch.nn.LogSoftmax(dim=1))

Nu vil vi definere standard PyTorch-træningsloop. Da vores datasæt er ret stort, vil vi til undervisningsformål kun træne i én epoch, og nogle gange endda i mindre end én epoch (angivelse af `epoch_size`-parameteren giver os mulighed for at begrænse træningen). Vi vil også rapportere akkumuleret træningsnøjagtighed under træningen; frekvensen for rapportering angives ved hjælp af `report_freq`-parameteren.


In [24]:
def train_epoch(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.NLLLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,features in dataloader:
        optimizer.zero_grad()
        out = net(features)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count

In [25]:
train_epoch(net,train_loader,epoch_size=15000)

3200: acc=0.8028125
6400: acc=0.8371875
9600: acc=0.8534375
12800: acc=0.85765625


(0.026090790722161722, 0.8620069296375267)

## BiGrams, TriGrams og N-Grams

En begrænsning ved en bag of words-tilgang er, at nogle ord indgår i udtryk med flere ord. For eksempel har ordet 'hot dog' en helt anden betydning end ordene 'hot' og 'dog' i andre sammenhænge. Hvis vi altid repræsenterer ordene 'hot' og 'dog' med de samme vektorer, kan det forvirre vores model.

For at løse dette bruges **N-gram-repræsentationer** ofte i metoder til dokumentklassifikation, hvor frekvensen af hvert ord, to-ords eller tre-ords udtryk er en nyttig funktion til at træne klassifikatorer. I bigram-repræsentation, for eksempel, tilføjer vi alle ordpar til ordforrådet, ud over de oprindelige ord.

Nedenfor er et eksempel på, hvordan man genererer en bigram bag of words-repræsentation ved hjælp af Scikit Learn:


In [26]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1)
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
bigram_vectorizer.fit_transform(corpus)
print("Vocabulary:\n",bigram_vectorizer.vocabulary_)
bigram_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()


Vocabulary:
 {'i': 7, 'like': 11, 'hot': 4, 'dogs': 2, 'i like': 8, 'like hot': 12, 'hot dogs': 5, 'the': 16, 'dog': 0, 'ran': 14, 'fast': 3, 'the dog': 17, 'dog ran': 1, 'ran fast': 15, 'its': 9, 'outside': 13, 'its hot': 10, 'hot outside': 6}


array([[1, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int64)

Den største ulempe ved N-gram-metoden er, at ordforrådet begynder at vokse ekstremt hurtigt. I praksis er vi nødt til at kombinere N-gram-repræsentation med nogle teknikker til reduktion af dimensioner, såsom *embeddings*, som vi vil diskutere i næste enhed.

For at bruge N-gram-repræsentation i vores **AG News**-datasæt, skal vi opbygge et specielt ngram-ordforråd:


In [27]:
counter = collections.Counter()
for (label, line) in train_dataset:
    l = tokenizer(line)
    counter.update(torchtext.data.utils.ngrams_iterator(l,ngrams=2))
    
bi_vocab = torchtext.vocab.vocab(counter, min_freq=1)

print("Bigram vocabulary length = ",len(bi_vocab))

Bigram vocabulary length =  1308842


Vi kunne derefter bruge den samme kode som ovenfor til at træne klassifikatoren, men det ville være meget hukommelsesineffektivt. I den næste enhed vil vi træne en bigram-klassifikator ved hjælp af embeddings.

> **Note:** Du kan kun beholde de ngrams, der forekommer i teksten mere end det angivne antal gange. Dette vil sikre, at sjældne bigrams udelades, og vil reducere dimensionaliteten betydeligt. For at gøre dette skal du indstille `min_freq`-parameteren til en højere værdi og observere ændringen i ordforrådets længde.


## Termfrekvens og Invers Dokumentfrekvens (TF-IDF)

I BoW-repræsentationen vægtes ordforekomster ens, uanset hvilket ord der er tale om. Det er dog tydeligt, at hyppige ord som *en*, *i* osv. er langt mindre vigtige for klassificeringen end specialiserede termer. Faktisk er nogle ord mere relevante end andre i de fleste NLP-opgaver.

**TF-IDF** står for **termfrekvens–invers dokumentfrekvens**. Det er en variation af bag of words, hvor man i stedet for en binær 0/1-værdi, der angiver, om et ord optræder i et dokument, bruger en flydende værdi, som relaterer sig til hyppigheden af ordets forekomst i korpuset.

Mere formelt defineres vægten $w_{ij}$ af et ord $i$ i dokumentet $j$ som:
$$
w_{ij} = tf_{ij}\times\log({N\over df_i})
$$
hvor
* $tf_{ij}$ er antallet af forekomster af $i$ i $j$, dvs. den BoW-værdi, vi har set tidligere
* $N$ er antallet af dokumenter i samlingen
* $df_i$ er antallet af dokumenter, der indeholder ordet $i$ i hele samlingen

TF-IDF-værdien $w_{ij}$ stiger proportionalt med, hvor mange gange et ord optræder i et dokument, og justeres i forhold til antallet af dokumenter i korpuset, der indeholder ordet. Dette hjælper med at tage højde for, at nogle ord optræder hyppigere end andre. For eksempel, hvis ordet optræder i *alle* dokumenter i samlingen, er $df_i=N$, og $w_{ij}=0$, og disse termer vil blive fuldstændigt ignoreret.

Du kan nemt oprette TF-IDF-vektorisering af tekst ved hjælp af Scikit Learn:


In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[0.43381609, 0.        , 0.43381609, 0.        , 0.65985664,
        0.43381609, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])

## Konklusion

Selvom TF-IDF-repræsentationer giver vægt til forskellige ord baseret på deres frekvens, er de ikke i stand til at repræsentere betydning eller rækkefølge. Som den berømte lingvist J. R. Firth sagde i 1935: "Den fulde betydning af et ord er altid kontekstuel, og ingen undersøgelse af betydning uden for kontekst kan tages seriøst." Senere i kurset vil vi lære, hvordan man fanger kontekstuel information fra tekst ved hjælp af sprogmodellering.



---

**Ansvarsfraskrivelse**:  
Dette dokument er blevet oversat ved hjælp af AI-oversættelsestjenesten [Co-op Translator](https://github.com/Azure/co-op-translator). Selvom vi bestræber os på at sikre nøjagtighed, skal du være opmærksom på, at automatiserede oversættelser kan indeholde fejl eller unøjagtigheder. Det originale dokument på dets oprindelige sprog bør betragtes som den autoritative kilde. For kritisk information anbefales professionel menneskelig oversættelse. Vi påtager os ikke ansvar for eventuelle misforståelser eller fejltolkninger, der måtte opstå som følge af brugen af denne oversættelse.
